##### Realizar a leitura dos arquivos .xlsx do BTG Pactual e desbloqueio com senha
01. Realizar for loop no Volume da landing layer
02. Abrir cada arquivo .xlsx com read binary - with open(path, 'rb') as file
03. Criar um dataframe em pandas
04. Converter para um dataframe em Spark
05. Salvar em na landing/unlocked em .csv e overwrite mode

In [0]:
display(dbutils.fs.ls('/Volumes/personalfinance/landing/vol_landing/btg/locked/'))

In [0]:
import msoffcrypto
import io
import pandas as pd

In [0]:
for file in dbutils.fs.ls('/Volumes/personalfinance/landing/vol_landing/btg/locked/'):
    file_name = file.name
    file_path = file.path
    file_date = file_name.split('_')[0]

    # Acessar arquivo protegido
    with open(file_path[5:], "rb") as file:
        office_file = msoffcrypto.OfficeFile(file)
        office_file.load_key(password="09939395620")

        decrypted = io.BytesIO()
        office_file.decrypt(decrypted)

    # Ler em pandas
    df_pd = pd.read_excel(decrypted)

    # Converter para Spark DataFrame
    df_spark = spark.createDataFrame(df_pd.astype(str))

    df_spark.write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv("/Volumes/personalfinance/landing/vol_landing/btg/unlocked/{}".format(file_date))